This file can be used to price Azure OpenAI and Azure AI Search for RAG-based use-cases.

Reference to the Azure Services:
1. Azure OpenAI : https://learn.microsoft.com/en-us/azure/ai-services/openai/overview
2. Azure AI Search : https://learn.microsoft.com/en-us/azure/search/search-what-is-azure-search
3. RAG Model : https://learn.microsoft.com/en-us/azure/search/retrieval-augmented-generation-overview

The prices below are based on Australia East Region.
Azure Pricing Calculator was used : https://azure.microsoft.com/en-au/pricing/calculator
The prices are in AUD and are subject to change.

In [388]:
# AI Search Storage Tier Costs per month
aiSearchStorageTierBasicCost = 148.76
aiSearchStorageTierStandardS1Cost = 496.62
aiSearchStorageTierStandardS2Cost = 1984.25
aiSearchStorageTierStandardS3Cost = 3972.97
aiSearchStorageTierStorageOptimizedL1Cost = 4293.99
aiSearchStorageTierStorageOptimizedL2Cost = 8586.85

def getAiSearchStorageTierCost(tier):
    if tier == "Basic":
        return aiSearchStorageTierBasicCost
    elif tier == "Standard S1":
        return aiSearchStorageTierStandardS1Cost
    elif tier == "Standard S2":
        return aiSearchStorageTierStandardS2Cost
    elif tier == "Standard S3":
        return aiSearchStorageTierStandardS3Cost
    elif tier == "Storage Optimized L1":
        return aiSearchStorageTierStorageOptimizedL1Cost
    elif tier == "Storage Optimized L2":
        return aiSearchStorageTierStorageOptimizedL2Cost
    else:
        return 0


# AI Search Semantic Ranking Costs per 1000 queries and first 1000 queries are free
aiSearchSemanticRankerCost = 1.53


In [389]:
# Azure OpenAI Cost based on GPT-3.5-Tier-0124-16K per 1000 tokens
azureOpenAIInputTokenCostGPT35Turbo16K = 0.0008
azureOpenAIOutputTokenCostGPT35Turbo16K = 0.0023

azureOpenAIInputTokenCostGPT4Turbo128K = 0.0153
azureOpenAIOutputTokenCostGPT4Turbo128K = 0.0460

azureOpenAIInputTokenCostGPT4TurboVision128K = 0.0153
azureOpenAIOutputTokenCostGPT4TurboVision128K = 0.0460

azureOpenAIInputTokenCostGPT48K = 0.0460
azureOpenAIOutputTokenCostGPT48K = 0.0919

azureOpenAIInputTokenCostGPT432K = 0.0919
azureOpenAIOutputTokenCostGPT432K = 0.1839

def getAzureOpenAIInputTokenCost(tier):
    if tier == "GPT-3.5-Turbo-16K":
        return azureOpenAIInputTokenCostGPT35Turbo16K
    elif tier == "GPT-4-Turbo-128K":
        return azureOpenAIInputTokenCostGPT4Turbo128K
    elif tier == "GPT-4-Turbo-Vision-128K":
        return azureOpenAIInputTokenCostGPT4TurboVision128K
    elif tier == "GPT-4-8K":
        return azureOpenAIInputTokenCostGPT48K
    elif tier == "GPT-4-32K":
        return azureOpenAIInputTokenCostGPT432K
    else:
        return 0
    

def getAzureOpenAIOutputTokenCost(tier):
    if tier == "GPT-3.5-Turbo-16K":
        return azureOpenAIOutputTokenCostGPT35Turbo16K
    elif tier == "GPT-4-Turbo-128K":
        return azureOpenAIOutputTokenCostGPT4Turbo128K
    elif tier == "GPT-4-Turbo-Vision-128K":
        return azureOpenAIOutputTokenCostGPT4TurboVision128K
    elif tier == "GPT-4-8K":
        return azureOpenAIOutputTokenCostGPT48K
    elif tier == "GPT-4-32K":
        return azureOpenAIOutputTokenCostGPT432K
    else:
        return 0



In this section we will start to model a contact center agent based RAG use-cases. The following are the parameters we will use

1. Number of agents e.g. 100
2. How much time it takes to address a customer query e.g. 5 mins
3. How many words used in an input query | Assumption 1 word = 0.75 tokens | https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
4. How many words in a query response | Assumption 1 word = 0.75 tokens | https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
5. working days in a month e.g. 20

In [390]:
numberOfUseCases = 15
numberOfAgents = 100
azureDiscount = 0.325
storageTier = "Standard S3"
aoaiModel = "GPT-3.5-Turbo-16K"
timeToAnswerCustomerQueryMins = 5
numbersOfQueriesToAssistACustomer = 2
wordsInputQuery = 2500
wordsOutputQuery = 500
workingHoursInADay = 8
workingDaysInMonth = 20
tokensInAWord=1.34

In [391]:
#perHourCost
costOfInputTokensPerHour = numberOfAgents * (60.0 / timeToAnswerCustomerQueryMins) * numbersOfQueriesToAssistACustomer * ((wordsInputQuery * tokensInAWord)/1000.0) * getAzureOpenAIInputTokenCost(aoaiModel)
costOfOutputTokensPerHour = numberOfAgents * (60.0 / timeToAnswerCustomerQueryMins) * numbersOfQueriesToAssistACustomer * ((wordsOutputQuery * tokensInAWord)/1000.0) * getAzureOpenAIOutputTokenCost(aoaiModel)
numberOfSearchQueriesPerHour = numberOfAgents * (60.0 / timeToAnswerCustomerQueryMins) * numbersOfQueriesToAssistACustomer

#perMonthCost
costOfInputTokensPerMonth = costOfInputTokensPerHour * workingHoursInADay * workingDaysInMonth
costOfOutputTokensPerMonth = costOfOutputTokensPerHour * workingHoursInADay * workingDaysInMonth
costNumberOfSearchQueriesPerMonth = (((numberOfSearchQueriesPerHour * workingHoursInADay * workingDaysInMonth)-1000)/1000) * aiSearchSemanticRankerCost
totalCostPerMonth = getAiSearchStorageTierCost(storageTier) + costOfInputTokensPerMonth + costOfOutputTokensPerMonth + costNumberOfSearchQueriesPerMonth

In [392]:
print("\n---------------Pricing Moodel Parameters--------------------")
print("Number of Use-Cases",numberOfUseCases)
print("Number of Customer Facing Agents",numberOfAgents)
print("How much time agent takes to service one customers (mins)", timeToAnswerCustomerQueryMins)
print("In serving a customer how many time agent query OpenAI model",numbersOfQueriesToAssistACustomer)
print("Words in the prompt Input",wordsInputQuery)
print("Words in the prompt Output",wordsOutputQuery)
print("Working hours in a day",workingHoursInADay)
print("Working days in a month",workingDaysInMonth)
print("Tokens in one word",tokensInAWord)
print("Azure Discount",azureDiscount)
print("Azure AI Search Storage Tier used",storageTier)
print("Azure OpenAI Model used",aoaiModel)


print("\n---------------OverAll Cost for One Use-Case--------------------")
print("Azure AI Search : Storage Tier Cost Per Month", getAiSearchStorageTierCost(storageTier))
print("Azure AI Search : Semantic Ranker Cost Per Month",costNumberOfSearchQueriesPerMonth)
print("Azure OpenAI : Input Token Cost Per Month",costOfInputTokensPerMonth)
print("Azure OpenAI : Output Token Cost Per Month",costOfOutputTokensPerMonth)
print("Total Azure AI Search Cost",getAiSearchStorageTierCost(storageTier) + costNumberOfSearchQueriesPerMonth)
print("Total Azure OpenAI Cost",costOfInputTokensPerMonth + costOfOutputTokensPerMonth)
print("Total Cost Per Month for One Use-Case",totalCostPerMonth)

print("\n---------------Multiple Similar Use-Cases--------------------")
print("Total cost for Azure AI Search for", numberOfUseCases, "UseCases", (getAiSearchStorageTierCost(storageTier) + costNumberOfSearchQueriesPerMonth) * numberOfUseCases)
print("Total cost for Azure OpenAI for", numberOfUseCases, "UseCases", (costOfInputTokensPerMonth + costOfOutputTokensPerMonth) * numberOfUseCases)
print("Total cost for", numberOfUseCases, "UseCases",numberOfUseCases*totalCostPerMonth)

print("\n---------------After Discount--------------------")
print("Total cost for Azure AI Search for", numberOfUseCases, "UseCases", ((getAiSearchStorageTierCost(storageTier) + costNumberOfSearchQueriesPerMonth) * numberOfUseCases) * (1-azureDiscount))
print("Total cost for Azure OpenAI for", numberOfUseCases, "UseCases", ((costOfInputTokensPerMonth + costOfOutputTokensPerMonth) * numberOfUseCases) * (1-azureDiscount))
print("Total cost for", numberOfUseCases, "UseCases",(numberOfUseCases*totalCostPerMonth) * (1 - azureDiscount))


---------------Pricing Moodel Parameters--------------------
Number of Use-Cases 10
Number of Customer Facing Agents 100
How much time agent takes to service one customers (mins) 5
In serving a customer how many time agent query OpenAI model 2
Words in the prompt Input 2500
Words in the prompt Output 500
Working hours in a day 8
Working days in a month 20
Tokens in one word 1.34
Azure Discount 0.325
Azure AI Search Storage Tier used Standard S3
Azure OpenAI Model used GPT-3.5-Turbo-16K

---------------OverAll Cost for One Use-Case--------------------
Azure AI Search : Storage Tier Cost Per Month 3972.97
Azure AI Search : Semantic Ranker Cost Per Month 585.99
Azure OpenAI : Input Token Cost Per Month 1029.1200000000001
Azure OpenAI : Output Token Cost Per Month 591.744
Total Azure AI Search Cost 4558.96
Total Azure OpenAI Cost 1620.864
Total Cost Per Month for One Use-Case 6179.824

---------------Multiple Similar Use-Cases--------------------
Total cost for Azure AI Search for 10 UseC